In [1]:
import numpy as np
import librosa
import librosa.display
import os
import matplotlib.pyplot as plt
import pyaudio
import time
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
%matplotlib inline

In [20]:
class melCNN(object):
    def __init__(self, sec=3, state=None, label=None):
        self.FORMAT = pyaudio.paFloat32
        self.SEC = sec
        self.STATE = state
        
        self.LABEL = label                
        self.labels = {0 : "doorbell",
                       1 : "fire_alarm",
                       2 : "hair_dry"}
        
        self.data_dir = os.path.dirname("data/")
        os.makedirs(self.data_dir, exist_ok=True)
        self.train_dir = os.path.join(self.data_dir, "train")
        os.makedirs(self.train_dir, exist_ok=True)        
        self.test_dir = os.path.join(self.data_dir, "test")
        os.makedirs(self.test_dir, exist_ok=True)
        if self.LABEL != None:
            self.label_dir = os.path.join(self.train_dir, self.labels[self.LABEL])
            os.makedirs(self.label_dir, exist_ok=True)
        
        self.CHANNELS = 1
        self.RATE = 44100
        self.n_fft = 1024
        self.hop_length = 1024
        self.n_mels = 128
        self.f_min = 20
        self.f_max = 8000
        
        self.count = 0
        self.data = None
        self.mel = None
        self.total_len = self.RATE * self.SEC
        self.total_data = np.zeros(self.total_len)
        
        self.pa = pyaudio.PyAudio()
            
        self.stream = self.pa.open(format=self.FORMAT,
                                   channels=self.CHANNELS,
                                   rate=self.RATE,
                                   input=True,
                                   output=False,
                                   frames_per_buffer=self.RATE)
        
        if self.STATE == None:
            pass
        else:
            self.loop()
        
    def loop(self):
        try:            
            while True:
                start = time.time()
                
                self.audioinput()
                self.pltmel()                
                
                end = time.time()
                print(str(round(end-start, 3))+"\tsec")
                self.count += 1
        except KeyboardInterrupt:
            self.pa.close()            
        
    def audioinput(self):
        for i in range(self.SEC):
            self.data = self.stream.read(self.RATE, exception_on_overflow=False)
            self.data = np.fromstring(self.data, np.float32)
            self.total_data[:-self.RATE] = self.total_data[self.RATE:]
            self.total_data[-self.RATE:] = self.data
            
    def pltmel(self):
        self.mel = librosa.feature.melspectrogram(y=self.total_data,
                                                  sr=self.RATE,
                                                  n_fft=self.n_fft,
                                                  hop_length=self.hop_length,
                                                  n_mels=self.n_mels,
                                                  power=1.0,
                                                  fmin=self.f_min,
                                                  fmax=self.f_max)
        
        plt.rcParams["figure.figsize"] = (2.24, 2.24)
        plt.axis("off")
        plt.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        #plt.imshow(librosa.power_to_db(self.mel, ref=np.max))        
            
        librosa.display.specshow(librosa.power_to_db(self.mel, ref=np.max), y_axis="mel", x_axis="time")
        
        if self.STATE == "save_data":
            plt.savefig(str(os.path.join(self.label_dir, "{:03}.jpg".format(self.count))), bbox_inches=None, pad_inches=0, dpi=100)
        elif self.STATE == "test":
            plt.savefig(str(os.path.join(self.label_dir, "{:03}.jpg".format(self.count))), bbox_inches=None, pad_inches=0, dpi=100)
        plt.clf()
        
    def train(self, useCNN=True, epochs=10, hidden=256, learning_rate=0.01):
        n_classes = len(self.labels)
        data = []
        label = []
        for i in os.listdir(self.train_dir):
            print(i)
            for k,v in mel.labels.items():                
                if i == v:                    
                    label_dir = os.path.join(self.train_dir, i)
                    images = os.listdir(label_dir)
                    for img in images:                
                        data.append(plt.imread(os.path.join(label_dir, img)))
                        label.append(k)
        data = np.array(data)
        label = np.array(label)
        data, label = shuffle(data, label)
        
        data = data / 255.0
        data = data.astype("float32")
        label = label.astype("float32")
        
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(data, label, test_size=0.3, random_state=0)

        if useCNN:
            self.model = keras.Sequential([    
                keras.layers.Conv2D(filters=30, kernel_size=(3,3), padding="valid", input_shape=(224, 224, 3)),
                keras.layers.Activation("relu"),
                keras.layers.MaxPooling2D(pool_size=(3,3)),
                keras.layers.Dropout(0.5),
                keras.layers.Conv2D(filters=30, kernel_size=(3, 3), padding="valid"),
                keras.layers.Activation("relu"),
                keras.layers.MaxPooling2D(pool_size=(3,3)),
                keras.layers.Flatten(),
                keras.layers.Dense(128, activation=tf.nn.relu),
                keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])
        else:
            self.model = keras.Sequential([
                keras.layers.Flatten(input_shape=(224, 224, 3)),
                keras.layers.Dense(hidden, activation=tf.nn.relu),
                keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])    
        self.model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])        
        
        self.train_history = self.model.fit(self.train_x, self.train_y,
                                            epochs=epochs,
                                            validation_data=(self.test_x, self.test_y))
        

In [21]:
'''
label 종류
1 : doorbell (3음)
2 : fire_alarm
3 : hair_dry
'''
# mel = melCNN(state="save_data", label=3)
mel = melCNN()

In [22]:
mel.train()

doorbell
fire_alarm
hair_dry
Train on 630 samples, validate on 270 samples
Epoch 1/10
630/630 [==============================] - 1s 2ms/sample - loss: 2.7541 - acc: 0.3667 - val_loss: 1.0890 - val_acc: 0.4222
Epoch 2/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.5706 - acc: 0.8127 - val_loss: 0.4064 - val_acc: 0.9444
Epoch 3/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.1226 - acc: 0.9746 - val_loss: 0.0844 - val_acc: 0.9889
Epoch 4/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.0831 - acc: 0.9778 - val_loss: 0.0713 - val_acc: 0.9889
Epoch 5/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.0613 - acc: 0.9778 - val_loss: 0.0694 - val_acc: 0.9889
Epoch 6/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.0661 - acc: 0.9810 - val_loss: 0.1071 - val_acc: 0.9704
Epoch 7/10
630/630 [==============================] - 1s 2ms/sample - loss: 0.0303 - acc: 0.9937 - val_loss: 0.0578 - val

In [23]:
mel.train_history.history['val_acc']

[0.42222223,
 0.9444444,
 0.98888886,
 0.98888886,
 0.98888886,
 0.97037035,
 0.98888886,
 0.98888886,
 0.9925926,
 0.98888886]